In [245]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/deep_learning_school/part_2/homework_2/
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive/deep_learning_school/part_2/homework_2/'
/content/drive/MyDrive/deep_learning_school/part_2/homework_2
 archive.zip		       training.1600000.processed.noemoticon.csv
'[homework]embeddings.ipynb'   word2vec.ipynb
 model.pt


<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

# Embeddings

Привет! В этом домашнем задании мы с помощью эмбеддингов решим задачу семантической классификации твитов.

Для этого мы воспользуемся предобученными эмбеддингами word2vec.

Для начала скачаем датасет для семантической классификации твитов:

In [ ]:
# !gdown https://drive.google.com/uc?id=1eE1FiUkXkcbw0McId4i7qY-L8hH-_Qph&export=download
# !unzip archive.zip

Downloading...
From: https://drive.google.com/uc?id=1eE1FiUkXkcbw0McId4i7qY-L8hH-_Qph
To: /content/drive/MyDrive/deep_learning_school/part_2/homework_2/archive.zip
84.9MB [00:01, 76.1MB/s]
Archive:  archive.zip
  inflating: training.1600000.processed.noemoticon.csv  


Импортируем нужные библиотеки:

In [246]:
import math
import random
import string

import numpy as np
import pandas as pd
import seaborn as sns

import torch
import nltk
import gensim
import gensim.downloader as api

In [3]:
random.seed(42)
np.random.seed(42)
torch.random.manual_seed(42)
torch.cuda.random.manual_seed(42)
torch.cuda.random.manual_seed_all(42)

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
data = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding="latin", header=None, names=["emotion", "id", "date", "flag", "user", "text"])

Посмотрим на данные:

In [ ]:
data.head()

,emotion,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


Выведем несколько примеров твитов, чтобы понимать, с чем мы имеем дело:

In [ ]:
examples = data["text"].sample(10)
print("\n".join(examples))

@Claire_Nelson i'm on the north devon coast the next few weeks  will be down in Devon again in may sometime i hope though!
@jhicks  i will think of you on Sunday!   Who are you by the way?  i am Val.
Out in the garden with the kids debating whether to cut grass, plant flowers, move stones or just do nothing 
@FrVerona thank u my love...u've shown me the light   fuck that...nothing would be left to the imagination there...
is with @jonasbrosfan1  going to buy LVATT together!!  everyone go buy it tomorrow!! 
follow @peterfacinelli and he will win a bet! team carlisle!! 
CLEANING!. Gosh darn it , but on the plus side i am finding quiet the treasure 
@Teddy_Salad Tears ok, cleansing of the soul &amp; just guess mine was needing some good laundrying LOL c now I'm ok &amp; hope u 2, Jody &amp;all 
@MissKellyO  morning, life is good, skint but the sun is out 
I miss school and my friends 


Как вилим, тексты твитов очень "грязные". Нужно предобработать датасет, прежде чем строить для него модель классификации.

Чтобы сравнивать различные методы обработки текста/модели/прочее, разделим датасет на dev(для обучения модели) и test(для получения качества модели).

In [5]:
indexes = np.arange(data.shape[0])
np.random.shuffle(indexes)
dev_size = math.ceil(data.shape[0] * 0.8)

dev_indexes = indexes[:dev_size]
test_indexes = indexes[dev_size:]

dev_data = data.iloc[dev_indexes]
test_data = data.iloc[test_indexes]

dev_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

## Обработка текста

Токенизируем текст, избавимся от знаков пунктуации и выкинем все слова, состоящие менее чем из 4 букв:

In [6]:
tokenizer = nltk.WordPunctTokenizer()
line = tokenizer.tokenize(dev_data["text"][0].lower())
print(" ".join(line))

@ chrishasboobs ahhh i hope your ok !!!


In [ ]:
filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
print(" ".join(filtered_line))

well done will making progress time


Загрузим предобученную модель эмбеддингов. 

Если хотите, можно попробовать другую. Полный список можно найти здесь: https://github.com/RaRe-Technologies/gensim-data.

Данная модель выдает эмбеддинги для **слов**. Строить по эмбеддингам слов эмбеддинги предложений мы будем ниже.

In [248]:
word2vec = api.load("glove-twitter-200")
# fasttext-wiki-news-subwords-300
# glove-twitter-200

[============--------------------------------------] 25.8% 247.1/958.4MB downloaded

KeyboardInterrupt: ignored

Нормализуем эмбеддинги, прежде чем обучать на них сеть. 
(наверное, вы помните, что нейронные сети гораздо лучше обучаются на нормализованных данных)

In [39]:
mean = np.mean(word2vec.vectors, 0)
std = np.std(word2vec.vectors, 0)
# norm_emb_line = [(word2vec.get_vector(w) - mean) / std for w in filtered_line if w in word2vec and len(w) > 3]
# print(sum(norm_emb_line).shape)
# print([all(norm_emb_line[i] == emb_line[i]) for i in range(len(emb_line))])

Сделаем датасет, который будет по запросу возвращать подготовленные данные.

In [35]:
from torch.utils.data import Dataset, random_split
import string
import re

class TwitterDataset(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        # Получи все токены из текста и профильтруй их
        tokens = tokenizer.tokenize(text.lower())
        tokens = [tok.translate(str.maketrans('', '', string.punctuation)) for tok in tokens ]
        tokens = [re.sub('\d', '', tok) for tok in tokens]
        tokens = [tok.lower() for tok in tokens if len(tok) > 1]
        return tokens

    def get_embeddings_(self, tokens):
        # embeddings = # Получи эмбеддинги слов и усредни их
        # embeddings = np.mean(np.array([(self.word2vec.get_vector(w) - mean) / std for w in tokens if w in self.word2vec and len(w) > 3]), axis=0)
        embeddings = np.array([(self.word2vec.get_vector(w) - mean) / std for w in tokens if w in self.word2vec and len(w) > 3])
        # print(embeddings)
        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [36]:
dev = TwitterDataset(dev_data, "text", "emotion", word2vec)

Отлично, мы готовы с помощью эмбеддингов слов превращать твиты в векторы и обучать нейронную сеть.

Превращать твиты в векторы, используя эмбеддинги слов, можно несколькими способами. А именно такими:

## Average embedding (2 балла)
---
Это самый простой вариант, как получить вектор предложения, используя векторные представления слов в предложении. А именно: вектор предложения есть средний вектор всех слов в предлоежнии (которые остались после токенизации и удаления коротких слов, конечно). 

In [40]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::1000]

examples = {"features": [np.mean(dev[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev[i]["target"] for i in example_indexes]}
print(len(examples["features"]))

1280


Давайте сделаем визуализацию полученных векторов твитов тренировочного (dev) датасета. Так мы увидим, насколько хорошо твиты с разными target значениями отделяются друг от друга, т.е. насколько хорошо усреднение эмбеддингов слов предложения передает информацию о предложении.

Для визуализации векторов надо получить их проекцию на плоскость. Сделаем это с помощью `PCA`. Если хотите, можете вместо PCA использовать TSNE: так у вас получится более точная проекция на плоскость (а значит, более информативная, т.е. отражающая реальное положение векторов твитов в пространстве). Но TSNE будет работать намного дольше.

In [41]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

pca = PCA(n_components=2)
# examples["transformed_features"] = # Обучи PCA на эмбеддингах слов
# pca.fit(examples['features'])
# examples['transformed_features'] = pca.fit_transform(examples['features'])
examples['transformed_features'] = TSNE(n_components=2).fit_transform(examples['features'])

In [ ]:
examples['transformed_features']

array([[  3.2897    ,  -0.32159203],
       [ 19.553543  , -15.548611  ],
       [-15.848399  ,  -6.745395  ],
       ...,
       [  0.3783207 ,  -5.654254  ],
       [ 11.677849  ,  -9.415133  ],
       [ -7.218142  ,  20.708246  ]], dtype=float32)

In [42]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

In [43]:
draw_vectors(
    examples["transformed_features"][:, 0], 
    examples["transformed_features"][:, 1], 
    color=[["red", "blue"][t] for t in examples["targets"]]
    )

Figure(id='1002', ...)

Скорее всего, на визуализации нет четкого разделения твитов между классами. Это значит, что по полученным нами векторам твитов не так-то просто определить, к какому классу твит пренадлежит. Значит, обычный линейный классификатор не очень хорошо справится с задачей. Надо будет делать глубокую (хотя бы два слоя) нейронную сеть.

Подготовим загрузчики данных.
Усреднее векторов будем делать в "батчевалке"(`collate_fn`). Она используется для того, чтобы собирать из данных `torch.Tensor` батчи, которые можно отправлять в модель.


In [45]:
from torch.utils.data import DataLoader


batch_size = 1024
num_workers = 8

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.FloatTensor(targets)}


train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

Определим функции для тренировки и теста модели:

In [46]:
from tqdm.notebook import tqdm


def training(model, optimizer, criterion, train_loader, epoch, device="cpu"):
    pbar = tqdm(train_loader, desc=f"Epoch {e + 1}. Train Loss: {0}")
    model.train()
    for batch in pbar:
        optimizer.zero_grad()
        features = batch["features"].to(device)
        targets = batch["targets"].to(device)

        # Получи предсказания модели
        # loss = ... # Посчитай лосс
        preds = model(features)
        preds = preds.squeeze(1)
        loss = criterion(preds, targets)

        loss.backward()
        optimizer.step()
        # Обнови параметры модели

        pbar.set_description(f"Epoch {e + 1}. Train Loss: {loss:.4}")
    

def testing(model, criterion, test_loader, device="cpu"):
    pbar = tqdm(test_loader, desc=f"Test Loss: {0}, Test Acc: {0}")
    mean_loss = 0
    mean_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in pbar:
            features = batch["features"].to(device)
            targets = batch["targets"].to(device)

            # Получи предсказания модели
            # loss = ... # Посчитай лосс
            # acc = ... # Посчитай точность модели
            preds = model(features)
            preds = preds.squeeze(1)
            # print(preds.shape, targets.shape)
            loss = criterion(preds, targets)
            preds = torch.round(preds)
            acc = (preds == targets).sum() / targets.shape[0]

            mean_loss += loss.item()
            mean_acc += acc.item()

            pbar.set_description(f"Test Loss: {loss:.4}, Test Acc: {acc:.4}")

    pbar.set_description(f"Test Loss: {mean_loss / len(test_loader):.4}, Test Acc: {mean_acc / len(test_loader):.4}")

    return {"Test Loss": mean_loss / len(test_loader), "Test Acc": mean_acc / len(test_loader)}

Создадим модель, оптимизатор и целевую функцию. Вы можете сами выбрать количество слоев в нейронной сети, ваш любимый оптимизатор и целевую функцию.


In [47]:
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module):
    def __init__(self, emb_size):
        super(Model, self).__init__()
        self.emb_size = emb_size

        self.layer_1 = nn.Linear(emb_size, emb_size*2) 
        self.layer_2 = nn.Linear(emb_size*2, emb_size*2)
        self.layer_out = nn.Linear(emb_size*2, 1) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm_1 = nn.BatchNorm1d(emb_size*2)
        self.batchnorm_2 = nn.BatchNorm1d(emb_size*2)

    def forward(self, x):

        x = F.relu(self.layer_1(x))
        x = self.batchnorm_1(x)
        x = F.relu(self.layer_2(x))
        x = self.batchnorm_2(x)
        x = torch.sigmoid(self.layer_out(x))

        return x

In [48]:
import torch.nn as nn
from torch.optim import Adam


# Не забудь поиграться с параметрами ;)
vector_size = dev.word2vec.vector_size
num_classes = 2
lr = 3e-4
num_epochs = 5

model = Model(vector_size)
model = model.cuda()
criterion = nn.BCELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, )


    

In [49]:
best_metric = 0.4964
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    # if log["Test Loss"] < best_metric:
    #     torch.save(model.state_dict(), "model.pt")
    #     best_metric = log["Test Loss"]


{'Test Loss': 0.494967066526413, 'Test Acc': 0.7567265625}



{'Test Loss': 0.4896845520734787, 'Test Acc': 0.75969921875}



{'Test Loss': 0.48612781322002413, 'Test Acc': 0.76212890625}



{'Test Loss': 0.48648548471927644, 'Test Acc': 0.762484375}



{'Test Loss': 0.48587766313552855, 'Test Acc': 0.762765625}


Наконец, обучим модель и протестируем её.

После каждой эпохи будем проверять качество модели на валидационной части датасета. Если метрика стала лучше, будем сохранять модель. **Подумайте, какая метрика (точность или лосс) будет лучше работать в этой задаче?** 

In [50]:
test_loader = DataLoader(
    TwitterDataset(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

# model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))


{'Test Loss': 0.4848500831058612, 'Test Acc': 0.7634909145367412}


In [ ]:
#{'Test Loss': 0.48611611403977145, 'Test Acc': 0.7634503544329073}

## Embeddings for unknown words (8 баллов)

Пока что использовалась не вся информация из текста. Часть информации фильтровалось – если слова не было в словаре эмбеддингов, то мы просто превращали слово в нулевой вектор. Хочется использовать информацию по-максимуму. Поэтому рассмотрим другие способы обработки слов, которых нет в словаре. А именно:

- Для каждого незнакомого слова будем запоминать его контекст(слова слева и справа от этого слова). Эмбеддингом нашего незнакомого слова будет сумма эмбеддингов всех слов из его контекста. (4 балла)
- Для каждого слова текста получим его эмбеддинг из Tfidf с помощью ```TfidfVectorizer``` из [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer). Итоговым эмбеддингом для каждого слова будет сумма двух эмбеддингов: предобученного и Tfidf-ного. Для слов, которых нет в словаре предобученных эмбеддингов, результирующий эмбеддинг будет просто полученный из Tfidf. (4 балла)

Реализуйте оба варианта **ниже**. Напишите, какой способ сработал лучше и ваши мысли, почему так получилось.

# ПРОБУЕМ СОСТАВИТЬ ЭМБЕДИНГ НА ОСНОВЕ ЕГО СОСЕДЕЙ(СКАЖИ КТО ТВОИ ДРУЗЬЯ И Я СКАЖУ КТО ТЫ)

In [51]:
def relu(x):
  if x > 0:
    return x
  else:
    return 0

In [122]:
from torch.utils.data import Dataset, random_split


class TwitterDatasetSurround(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec, window: int):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

        self.window = window

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        # Получи все токены из текста и профильтруй их
        
        tokens = tokenizer.tokenize(text.lower())
        tokens = [tok.translate(str.maketrans('', '', string.punctuation)) for tok in tokens ]
        tokens = [re.sub('\d', '', tok) for tok in tokens]
        tokens = [tok for tok in tokens if len(tok) > 1]
        return tokens

    def get_embeddings_(self, tokens):
        embeddings = [0]* len(tokens)

        for i, tok in enumerate(tokens):
          if tok in self.word2vec:
            embeddings[i] = self.word2vec[tok]

        for i in range(1, len(embeddings)):
          emb = embeddings[i]
          if type(emb) == int:
            window_embs = np.mean(np.array(
                [e for e in embeddings[relu(i-self.window):(i+self.window)]]), axis=0)
            embeddings[i] = window_embs
        
        embeddings = np.array([(e - self.mean) / self.std for e in embeddings])
        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [54]:
dev = TwitterDatasetSurround(dev_data, "text", "emotion", word2vec, window=2)

In [55]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::1000]

examples = {"features": [np.mean(dev[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev[i]["target"] for i in example_indexes]}
print(len(examples["features"]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


1280


In [56]:
from torch.utils.data import DataLoader


batch_size = 1024
num_workers = 5

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.FloatTensor(targets)}


train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

In [ ]:
len(train)

1024000

In [57]:
import torch.nn as nn
from torch.optim import Adam


# Не забудь поиграться с параметрами ;)
vector_size = dev.word2vec.vector_size
num_classes = 2
lr = 2e-3
num_epochs = 5

model = Model(vector_size)
model = model.cuda()
criterion = nn.BCELoss()
optimizer = torch.optim.AdamW(model.parameters(),  lr=lr, )


In [58]:
best_metric = 0.4323
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local


{'Test Loss': 0.4532021661996841, 'Test Acc': 0.782703125}


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local


{'Test Loss': 0.4401737867593765, 'Test Acc': 0.7918125}


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local


{'Test Loss': 0.4380738227367401, 'Test Acc': 0.79263671875}


In [60]:
test_loader = DataLoader(
    TwitterDatasetSurround(test_data, "text", "emotion", word2vec, window=2), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

# model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local


{'Test Loss': 0.4384909149366446, 'Test Acc': 0.7927066693290735}


# ПРОБУЕМ ИСПОЛЬЗОВАТЬ TF-IDF

In [249]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

tf_idf_vect = TfidfVectorizer(max_features=110000)


In [250]:
def filter_sent(text):
  text = str(text)
  text = text.translate(str.maketrans('', '', string.punctuation))
  text = re.sub('\d', '', text)
  text = text.lower()
  return text

In [251]:

dev_data['text'] = dev_data['text'][:128000].apply(filter_sent)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [252]:
tf_idf_vect_train_matrix = tf_idf_vect.fit_transform(dev_data['text'][:128000].tolist(), ) 


In [253]:
tf_idf_vect_train_matrix = tf_idf_vect_train_matrix.T

In [ ]:
from sklearn.decomposition import TruncatedSVD

reduced_tf_idf_train = TruncatedSVD(n_components=200).fit_transform(tf_idf_vect_train_matrix)

In [145]:
tf_idf_words_dict = {tok:i for i, tok in enumerate(tf_idf_vect.get_feature_names())}

In [237]:
from torch.utils.data import Dataset, random_split


class TwitterDatasetTfIdf(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, 
                 word2vec: gensim.models.Word2Vec, 
                 tf_idf_matrix = reduced_tf_idf_train, 
                 tf_idf_words = tf_idf_words_dict):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec
        self.tf_idf_matrix = (tf_idf_matrix - np.mean(tf_idf_matrix, axis=0)) / np.std(tf_idf_matrix, axis=0)
        self.tf_idf_words = tf_idf_words

        self.label2num = lambda label: 0 if label == 0 else 1

        self.mean_word2vec = np.mean(word2vec.vectors, axis=0)
        self.std_word2vec = np.std(word2vec.vectors, axis=0)

        self.mean_tf_idf = np.mean(tf_idf_matrix, axis=0)
        self.std_tf_idf = np.std(tf_idf_matrix, axis=0)


    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        # Получи все токены из текста и профильтруй их
        # return [w for w in tokenizer.tokenize(text.lower()) if all(c not in string.punctuation for c in w) ] 
        tokens = tokenizer.tokenize(text.lower())
        tokens = [tok.translate(str.maketrans('', '', string.punctuation)) for tok in tokens ]
        tokens = [re.sub('\d', '', tok) for tok in tokens]
        # tokens = [tok for tok in tokens if len(tok.strip)]
        # if len(tokens) == 0:
        #   print("ZERO!!"*100)
        return tokens

    def get_embeddings_(self, tokens):
        embeddings = [0]* len(tokens)

        invalid_positions = []

        for i, tok in enumerate(tokens):
          if tok in self.word2vec:
            emb_word2vec = self.word2vec[tok]
            emb_word2vec = (emb_word2vec - self.mean_word2vec) / self.std_word2vec

            emb_tf_idf_exists = False 
            if tok in self.tf_idf_words:
              emb_position = self.tf_idf_words.get(tok)
              emb_tf_idf = self.tf_idf_matrix[emb_position]
              # emb_tf_idf = (emb_tf_idf - self.mean_tf_idf) / self.std_tf_idf
              emb_tf_idf_exists = True
            if emb_tf_idf_exists:
              embeddings[i] = (emb_word2vec + emb_tf_idf) / 2
            else:
              embeddings[i] = emb_word2vec
              invalid_positions.append(i)


        for i in invalid_positions:
          token = tokens[i]
          if token in self.tf_idf_words:
            emb_position = self.tf_idf_words.get(token)
            tf_idf_emb = self.tf_idf_matrix[emb_position]
            # tf_idf_emb = (tf_idf_emb - self.mean_tf_idf) / self.std_tf_idf
            embeddings[i] = tf_idf_emb

        embeddings = np.array([emb for emb in embeddings if type(emb) != int])

        if len(embeddings) == 0:
          embeddings = np.array([[0] * 200])
          # print(embeddings.shape, embeddings,"ZERO!!"*10)
        # else:
        #     embeddings = np.array(embeddings)
        #     if len(embeddings.shape) == 1:
        #         embeddings = embeddings.reshape(-1, 1)

        
        # embeddings = np.array([(e - self.mean_word2vec) / self.std_word2vec for e in embeddings])
          
        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [106]:
type(False)

bool

In [239]:
import gc
gc.collect()

415

In [238]:
dev = TwitterDatasetTfIdf(dev_data, "text", "emotion", word2vec, )

In [240]:
from torch.utils.data import DataLoader


batch_size = 1024
num_workers = 5

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.FloatTensor(targets)}


train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

In [241]:
next(iter(train_loader))

{'features': tensor([[ 8.1622, 22.3391,  8.8255,  ...,  0.5930, -0.3359, -0.2295],
         [16.1242, -1.1279, -0.8709,  ...,  1.1804, -0.9792, -0.6437],
         [ 9.7354, -2.1915, -0.7622,  ..., -0.9892,  0.9993, -2.2555],
         ...,
         [10.6739, -5.6554,  7.8396,  ..., -1.2767, -1.7263, -0.6689],
         [17.2703,  4.7131, 11.9533,  ..., -1.5934,  0.7186,  0.6167],
         [12.6172,  1.1930, -9.4236,  ..., -0.9399, -2.2558,  1.3328]]),
 'targets': tensor([1., 0., 0.,  ..., 1., 1., 0.])}

In [242]:
import torch.nn as nn
from torch.optim import Adam


# Не забудь поиграться с параметрами ;)
vector_size = dev.word2vec.vector_size
num_classes = 2
lr = 2e-3
num_epochs = 5

model = Model(vector_size)
model = model.cuda()
criterion = nn.BCELoss()
# lr=lr,
optimizer = torch.optim.AdamW(model.parameters(),  lr=lr, )


In [243]:
best_metric = 0.4323
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)


{'Test Loss': 0.4927272437810898, 'Test Acc': 0.7566484375}



{'Test Loss': 0.48638978266716004, 'Test Acc': 0.76083203125}



{'Test Loss': 0.481009969830513, 'Test Acc': 0.76512109375}



{'Test Loss': 0.4777219401597977, 'Test Acc': 0.76682421875}



{'Test Loss': 0.47542602264881134, 'Test Acc': 0.76959765625}


KeyboardInterrupt: ignored

In [244]:
test_loader = DataLoader(
    TwitterDatasetTfIdf(test_data, "text", "emotion", word2vec,), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

# model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))


{'Test Loss': 0.4896249985352111, 'Test Acc': 0.759185303514377}


# Напишите, какой способ сработал лучше и ваши мысли, почему так получилось.

Способ по вычленению неизвестного слова, полагаясь на его окружающие сработал лучше. Во-первых потому что это естественное его получение в концепции word2vec. То есть мы по факту получаем вектор UNK слова из той же размерности что и другие известные, поэтому он лучше способен закодировать смысл нашего слова.

tf_idf подход сработал намного хуже. 
Во-первых потому что мы пытаемся как-то работать с векторами из разных пространств и распределений и в итоге вектора получаются довольно шумными.\

Во-вторых этот подход становится очень непрактичным, когда документов и слов становится очень много. Если мы добавим новый документ это заново все пересчитывать, фильтровать, сокращать размерность, соответсвенно терять часть информации. не очень удобно (имхо). По этой причине я не смог задействовать весь tftdf, а ограничиться 100к документами.

Да и вообще на самом деле лучше делать словарь из частей слов, чтобы сократить размерность словаря, да и это более естественно для нас, когда мы не знаем какое-то слово мы смотрим на его "приставку", "корень", "окончание" и пытаемся выцепить из них что-то похожее что мы видели раньше.

Окружающие слова: loss=0.438, 'Test Acc': 0.792 (колеблется от 0.791 - 0.796)

Способ с Tfidf: loss=0.489, 'Test Acc': 0.759

Loss для этой задачи более показателен, потому что он учитывает степень уверенности в ответе скажем чем она больше, тем меньше loss. С  accuracy такого нет, там все что меньше 0.5 считается неправильным, что не верно. А еще желательно конечно учитывать баланс классов, но это другая история...